## Final Project
### Data Science LHL Project: Restaurant Reccomenation system

### Part 3: Building Model

By: Chloe Phuong

In [1047]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import string
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
import re
from nltk.stem import WordNetLemmatizer

In [1048]:
final_restrev_df = pd.read_json('final_restrev_df')

# Display the DataFrame
final_restrev_df

,business_id,rest_name,address,city,postal_code,latitude,longitude,rest_avg_stars,total_reviews,is_open,categories,review_id,user_id,review_stars,review_text
0,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5,"Incredible, the chicken is so tender, seasonin..."
1,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4,We were able to check this place out yesterday...
2,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,w56U5WwwCzd2Sz9aXsuXjQ,XbiKsujS_qxU3xsr0xUqmQ,4,I think this will be our go-to before any even...
3,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5,"Incredible, the chicken is so tender, seasonin..."
4,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4,We were able to check this place out yesterday...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,ZcWCQcO_eKKZ0Z5mdY6K-g,ykWqtFWldi0bojZ6rorFgg,1,This is one of our favourite go to-s in the ne...
5576,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,xTaL-8592nVx7mjYKz1f8g,C282JQOYkRWeAjAcfKlK1g,4,Great spot in Parkdale serving French course m...
5577,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,-bZ8wE7TaRDkMuBeLM91zQ,RD365MaKbCCjpZQ2KkxROg,5,This felt just like a typical Parisian restaur...
5578,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,ZcWCQcO_eKKZ0Z5mdY6K-g,ykWqtFWldi0bojZ6rorFgg,1,This is one of our favourite go to-s in the ne...


In [1049]:
final_restrev_df.to_csv('final_restrev_df.csv', index=False)

In [1050]:
final_restrev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5580 entries, 0 to 5579
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   business_id     5580 non-null   object 
 1   rest_name       5580 non-null   object 
 2   address         5580 non-null   object 
 3   city            5580 non-null   object 
 4   postal_code     5580 non-null   object 
 5   latitude        5580 non-null   float64
 6   longitude       5580 non-null   float64
 7   rest_avg_stars  5580 non-null   float64
 8   total_reviews   5580 non-null   int64  
 9   is_open         5580 non-null   bool   
 10  categories      5580 non-null   object 
 11  review_id       5580 non-null   object 
 12  user_id         5580 non-null   object 
 13  review_stars    5580 non-null   int64  
 14  review_text     5580 non-null   object 
dtypes: bool(1), float64(3), int64(2), object(9)
memory usage: 659.4+ KB


In [1051]:
# Lowercasing
final_restrev_df['categories'] = final_restrev_df['categories'].str.lower()

# Removal of Punctuation
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    '''Custom function to remove punctuation'''
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

final_restrev_df['categories'] = final_restrev_df['categories'].apply(lambda text: remove_punctuation(text))

# Removal of Stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    '''Custom function to remove stopwords'''
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

final_restrev_df['categories'] = final_restrev_df['categories'].apply(lambda text: remove_stopwords(text))

In [1052]:
# Lower Casing
final_restrev_df['review_text'] = final_restrev_df['review_text'].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    '''custom function to remove the punctuation'''
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

final_restrev_df['review_text'] = final_restrev_df['review_text'].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    '''custom function to remove the stopwords'''
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

final_restrev_df['review_text'] = final_restrev_df['review_text'].apply(lambda text: remove_stopwords(text))

In [1053]:
# RESTAURANT NAMES:
restaurant_names = list(final_restrev_df['rest_name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

In [1062]:
def restaurant_recommendations(rest_name, rating_tolerance):

    #Setting assertions on the input parameters data types
    assert all(isinstance(rest_name, str) for rest_name in rest_name), 'food_category elements should be strings'
    assert isinstance(rating_tolerance, float), 'rating_tolerance should be a float (i.e. number with a decimal)'

    #Setting an assertion on the rating_tolerance to only accept an input, difference threshold, between 1.0 and 4.0 given the Yelp rating scale of 1 through 5
    assert 1.0 <= rating_tolerance <= 4.5, 'rating_tolerance should be between 1.0 and 4.5, given the Yelp rating scale of 1 through 5'

    # Filter restaurants based on rating tolerance
    filtered_restaurants = final_restrev_df[final_restrev_df['rest_avg_stars'] >= rating_tolerance]

    # Drop duplicates based on rest_name
    filtered_restaurants = filtered_restaurants.drop_duplicates(subset='rest_name', keep='first').reset_index(drop=True)

    # Find the restaurant with the provided rest_name
    target_restaurant = filtered_restaurants[filtered_restaurants['rest_name'] == rest_name]

    if target_restaurant.empty:
        return "The provided restaurant name is not found in the dataset."

    # Calculate the similarity scores based on categories
    category_vectorizer = TfidfVectorizer()
    category_matrix = category_vectorizer.fit_transform(filtered_restaurants['categories'])
    category_similarity = cosine_similarity(category_matrix)

    # Calculate the similarity scores based on review text
    review_vectorizer = TfidfVectorizer()
    review_matrix = review_vectorizer.fit_transform(filtered_restaurants['review_text'])
    review_similarity = cosine_similarity(review_matrix)

    # Combine the similarity scores with a weight of 7:3 (categories:review_text)
    combined_similarity = 0.7 * category_similarity + 0.3 * review_similarity

    # Get the index of the target restaurant within the filtered restaurants dataframe
    rest_index = filtered_restaurants[filtered_restaurants['rest_name'] == rest_name].index[0]

    # Get the similarity scores for all restaurants
    rest_scores = combined_similarity[:, rest_index]

    # Sort the similarity scores
    sorted_indices = rest_scores.argsort()[::-1]

    # Get the top 15 recommended restaurants (excluding the target restaurant itself)
    top_recommendations = filtered_restaurants.iloc[sorted_indices]
    top_recommendations = top_recommendations[top_recommendations.index != rest_index][:15]

    recommended_rest = top_recommendations[['rest_name', 'address', 'rest_avg_stars', 'total_reviews', 'categories', 'review_text']].sort_values(by='rest_avg_stars', ascending=False)

    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR CATEGORIES AND REVIEWS: ' % (str(len(recommended_rest)), rest_name))

    return recommended_rest

# Example usage
rest_name = 'Richmond Station'
rating_tolerance = 4.5

recommendations = restaurant_recommendations(rest_name, rating_tolerance)
recommendations

TOP 15 RESTAURANTS LIKE Richmond Station WITH SIMILAR CATEGORIES AND REVIEWS: 


,rest_name,address,rest_avg_stars,total_reviews,categories,review_text
84,Bistro Five61,"561 Avenue Road, Toronto, ON M4V 2K4, Canada",5.0,4,canadian new american traditional,excellent dinner last week small quiet unassum...
81,Megumi Mazesoba,"46 Saint Clair Avenue E, Toronto, ON M4T 1M9, ...",5.0,24,japanese,first time ever hearing brothless ramen wife t...
82,Mary Be Kitchen,"21B St. Clair Avenue W, Toronto, ON M4V 1K6, C...",4.5,72,canadian new,love place especially options food preference ...
130,Grey Gardens,"199 Augusta Avenue, Toronto, ON M5T 2L4, Canada",4.5,128,wine bars canadian new,grey gardens mustvisit restaurant anyone looki...
111,The Hole In the Wall,"2867 Dundas Street W, Toronto, ON M6P 1Y9, Canada",4.5,172,pubs canadian new,love place food cocktails service excellent so...
40,Urban House,"718 Queen Street E, Toronto, ON M4M 1H2, Canada",4.5,13,pubs canadian new,place good believe popular pandemic seeking lo...
98,Her Father's Cider Bar & Kitchen,"119 Harbord Street, Toronto, ON M5S 1G7, Canada",4.5,189,gastropubs bars canadian new,went whim drink bartender jess absolutely made...
94,Rasa,"196 Robert Street, Toronto, ON M5S 2K8, Canada",4.5,322,canadian new tapassmall plates,absolutely delicious toronto weekend glad kept...
118,Yeah Yeah Thai,"382 Spadina Avenue, Toronto, ON M5T 2G5, Canada",4.5,17,thai hong kong style cafe,absolutely loved place must try gem chinatown ...
103,Thai Nyyom,"1419 Bloor Street W, Toronto, ON M6P 3L4, Canada",4.5,22,thai,place tiny restaurant tiny menu menu basically...


In [1055]:
import pickle

# Save the model
model = restaurant_recommendations
with open('restaurant_recommendations.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the model
with open('restaurant_recommendations.pkl', 'rb') as f:
    loaded_model = pickle.load(f)